# Yelp API
---

Author: Jackson Muehlbauer

Date: 2/26/2023

In [1]:
# imports
import json
import pandas as pd
from yelpapi import YelpAPI

In [2]:
with open('/Users/jacksonmuehlbauer/.secret/yelp_api.json') as f:
    login = json.load(f)

login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
# API call

search_results = yelp_api.search_query(location = 'Minneapolis', term = 'ramen')

print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [5]:
# How many results
search_results['total']

240

In [6]:
# Make DataFrame of the businesses

biz = pd.DataFrame(search_results['businesses'])
biz.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,94ciL-h9yS6pm0BOFzjLEA,kyuramen-eagan-eagan,Kyuramen - Eagan,https://s3-media1.fl.yelpcdn.com/bphoto/DQXmRt...,False,https://www.yelp.com/biz/kyuramen-eagan-eagan?...,71,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.0,"{'latitude': 44.83054, 'longitude': -93.16147}","[restaurant_reservation, delivery, pickup]",$$,"{'address1': '1278 Town Centre Dr', 'address2'...",+19526001032,(952) 600-1032,18105.276758
1,xoq8XRgV5f031ZI3aHrSFg,ramen-kazama-minneapolis,Ramen Kazama,https://s3-media1.fl.yelpcdn.com/bphoto/Bbal0I...,False,https://www.yelp.com/biz/ramen-kazama-minneapo...,385,"[{'alias': 'ramen', 'title': 'Ramen'}]",4.0,"{'latitude': 44.9410781860352, 'longitude': -9...","[delivery, pickup]",$$,"{'address1': '3400 Nicollet Ave S', 'address2'...",+16123536160,(612) 353-6160,2884.902824


In [7]:
biz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             20 non-null     object 
 1   alias          20 non-null     object 
 2   name           20 non-null     object 
 3   image_url      20 non-null     object 
 4   is_closed      20 non-null     bool   
 5   url            20 non-null     object 
 6   review_count   20 non-null     int64  
 7   categories     20 non-null     object 
 8   rating         20 non-null     float64
 9   coordinates    20 non-null     object 
 10  transactions   20 non-null     object 
 11  price          13 non-null     object 
 12  location       20 non-null     object 
 13  phone          20 non-null     object 
 14  display_phone  20 non-null     object 
 15  distance       20 non-null     float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 2.5+ KB


In [8]:
# Get second page of businesses
search_20 = yelp_api.search_query(location = 'Minneapolis', term = 'ramen', offset = 20)

# Add to dataframe
biz20 = pd.DataFrame(search_20['businesses'])
biz20.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,KAgI00gIx_t5AXhCs3x9BQ,ichiddo-ramen-bloomington,Ichiddo Ramen,https://s3-media4.fl.yelpcdn.com/bphoto/vOdUt8...,False,https://www.yelp.com/biz/ichiddo-ramen-bloomin...,17,"[{'alias': 'ramen', 'title': 'Ramen'}]",3.5,"{'latitude': 44.855983332522186, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '322 Central Pkwy', 'address2': '...",+19526812358,(952) 681-2358,12759.968085
1,Qfi2-C45fk48c0JG4gVXbA,young-man-minneapolis,Young Man,https://s3-media1.fl.yelpcdn.com/bphoto/VJJ-TK...,False,https://www.yelp.com/biz/young-man-minneapolis...,53,"[{'alias': 'indonesian', 'title': 'Indonesian'...",5.0,"{'latitude': 44.934536, 'longitude': -93.278038}",[delivery],$$,"{'address1': '3752 Nicollet Ave', 'address2': ...",+19294442705,(929) 444-2705,3586.981581


In [10]:
# concatenate the dataframes

df = pd.concat([biz, biz20], axis = 0, ignore_index = True)
df.shape

(40, 16)

In [12]:
# Make the businesses dataframe as a records-oriented JSON file

biz_json = df.to_json(orient = 'records')

In [13]:
# Save the JSON file

with open('minneapolis_ramen.json', 'w') as f:
    json.dump(biz_json, f)